# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6d774f2d60>
├── 'a' --> tensor([[ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215]])
└── 'x' --> <FastTreeValue 0x7f6d774f2f70>
    └── 'c' --> tensor([[ 0.9110,  0.3386,  0.2791, -0.1886],
                        [-0.9278, -0.5273,  0.2357,  0.1806],
                        [-1.6056,  0.8934, -1.0785,  1.7385]])

In [4]:
t.a

tensor([[ 1.0330,  0.5842,  0.4965],
        [ 0.1226, -0.4733,  0.3215]])

In [5]:
%timeit t.a

66.1 ns ± 1.34 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6d774f2d60>
├── 'a' --> tensor([[-1.2274, -1.8933,  0.1946],
│                   [-1.1886, -0.1864, -0.7520]])
└── 'x' --> <FastTreeValue 0x7f6d774f2f70>
    └── 'c' --> tensor([[ 0.9110,  0.3386,  0.2791, -0.1886],
                        [-0.9278, -0.5273,  0.2357,  0.1806],
                        [-1.6056,  0.8934, -1.0785,  1.7385]])

In [7]:
%timeit t.a = new_value

75.7 ns ± 2.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215]]),
    x: Batch(
           c: tensor([[ 0.9110,  0.3386,  0.2791, -0.1886],
                      [-0.9278, -0.5273,  0.2357,  0.1806],
                      [-1.6056,  0.8934, -1.0785,  1.7385]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0330,  0.5842,  0.4965],
        [ 0.1226, -0.4733,  0.3215]])

In [11]:
%timeit b.a

66 ns ± 1.01 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2636,  0.5299, -0.6454],
               [-0.1654,  0.7859,  1.1730]]),
    x: Batch(
           c: tensor([[ 0.9110,  0.3386,  0.2791, -0.1886],
                      [-0.9278, -0.5273,  0.2357,  0.1806],
                      [-1.6056,  0.8934, -1.0785,  1.7385]]),
       ),
)

In [13]:
%timeit b.a = new_value

575 ns ± 23.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

933 ns ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

257 µs ± 5.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

248 µs ± 9.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6cbd715df0>
├── 'a' --> tensor([[[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]],
│           
│                   [[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]],
│           
│                   [[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]],
│           
│                   [[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]],
│           
│                   [[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]],
│           
│                   [[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]],
│           
│                   [[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]],
│           
│                   [[ 1.0330,  0.5842,  0.4965],
│                    [ 0.1226, -0.4733,  0.3215]]])
└── 'x' --> <FastTreeValue 0x7f6cbd715dc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.4 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6cbd715130>
├── 'a' --> tensor([[ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215],
│                   [ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215],
│                   [ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215],
│                   [ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215],
│                   [ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215],
│                   [ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215],
│                   [ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215],
│                   [ 1.0330,  0.5842,  0.4965],
│                   [ 0.1226, -0.4733,  0.3215]])
└── 'x' --> <FastTreeValue 0x7f6cc4678ac0>
    └── 'c' --> tensor([[ 0.9110,  0.3386,  0.2791, -0.1886],
                        [-0.9278, -0.5273,  0.2357,  0.1806],
                 

In [23]:
%timeit t_cat(trees)

41.5 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.7 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]],
       
               [[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]],
       
               [[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]],
       
               [[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]],
       
               [[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]],
       
               [[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]],
       
               [[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]],
       
               [[ 1.0330,  0.5842,  0.4965],
                [ 0.1226, -0.4733,  0.3215]]]),
    x: Batch(
           c: tensor([[[ 0.9110,  0.3386,  0.2791, -0.1886],
                       [-0.9278, -0.5273,  0.2357,  0.1806],
                       [-1.6056,  0.8934, -1.0785,  1.7385]],
         

In [26]:
%timeit Batch.stack(batches)

99.3 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215],
               [ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215],
               [ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215],
               [ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215],
               [ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215],
               [ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215],
               [ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215],
               [ 1.0330,  0.5842,  0.4965],
               [ 0.1226, -0.4733,  0.3215]]),
    x: Batch(
           c: tensor([[ 0.9110,  0.3386,  0.2791, -0.1886],
                      [-0.9278, -0.5273,  0.2357,  0.1806],
                      [-1.6056,  0.8934, -1.0785,  1.7385],
                      [ 0.9110,  0.3386,  0.2791, -0.1886],
                      [-0.9278, -

In [28]:
%timeit Batch.cat(batches)

182 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

535 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
